In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [2]:
DATA_PATH = "/home/rohit/Desktop/DL/text_data"

In [3]:
def load_data():
    with open(DATA_PATH, "r") as f:
        data = json.load(f)
    
    X = np.array(data["mfcc"])
    y = np.array(data["label"])
    return X, y

In [4]:
def prepare_data(test_size, validation_size):
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test
    
    

In [5]:
def build_model(input_shape):
    #create model
    model = keras.Sequential()
    
    #1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    #2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    #3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())
    
    #flatten and feed into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    #output layer
    model.add(keras.layers.Dense(2, activation='softmax'))
    
    return model

In [7]:
# create datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_data(0.25, 0.2)

In [8]:
# build cnn net

input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape)

#compile the network

optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
             loss="sparse_categorical_crossentropy",
             metrics=['accuracy'])

# train the cnn

model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

# evaluate cnn on test set

test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy on test set is: {}'.format(test_accuracy))

# make prediction on sample

Epoch 1/30
46/46 [==============================] - 3s 54ms/step - loss: 0.6273 - accuracy: 0.6882 - val_loss: 0.4651 - val_accuracy: 0.8430
Epoch 2/30
46/46 [==============================] - 2s 42ms/step - loss: 0.4739 - accuracy: 0.8125 - val_loss: 0.4525 - val_accuracy: 0.8237
Epoch 3/30
46/46 [==============================] - 2s 43ms/step - loss: 0.4492 - accuracy: 0.8080 - val_loss: 0.4383 - val_accuracy: 0.8044
Epoch 4/30
46/46 [==============================] - 2s 42ms/step - loss: 0.3822 - accuracy: 0.8380 - val_loss: 0.4164 - val_accuracy: 0.8209
Epoch 5/30
46/46 [==============================] - 2s 41ms/step - loss: 0.3816 - accuracy: 0.8362 - val_loss: 0.3751 - val_accuracy: 0.8430
Epoch 6/30
46/46 [==============================] - 2s 44ms/step - loss: 0.3390 - accuracy: 0.8755 - val_loss: 0.3431 - val_accuracy: 0.8540
Epoch 7/30
46/46 [==============================] - 3s 55ms/step - loss: 0.2942 - accuracy: 0.8720 - val_loss: 0.3446 - val_accuracy: 0.8402
Epoch 8/30
46